In [1]:
from tqdm import tqdm
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings 
warnings.filterwarnings('ignore')
data=pd.read_csv("data/train_data.csv")

In [2]:
print(data.columns)
print(data.shape)

Index(['HADM_ID', 'SUBJECT_ID', 'GENDER', 'AGE', 'Admission Weight (Kg)',
       'Height (cm)', 'Heart Rate', 'Non Invasive Blood Pressure diastolic',
       'Non Invasive Blood Pressure mean',
       'Non Invasive Blood Pressure systolic', 'Respiratory Rate', 'stroke',
       'Tachycardia', 'Bradycardia', 'Hypertension', 'Hypotension',
       'Tachypnea', 'Bradypnea'],
      dtype='object')
(207408, 18)


In [3]:
new_columns = [col for col in data.columns if col != 'stroke'] + ['stroke']
print(new_columns)

['HADM_ID', 'SUBJECT_ID', 'GENDER', 'AGE', 'Admission Weight (Kg)', 'Height (cm)', 'Heart Rate', 'Non Invasive Blood Pressure diastolic', 'Non Invasive Blood Pressure mean', 'Non Invasive Blood Pressure systolic', 'Respiratory Rate', 'Tachycardia', 'Bradycardia', 'Hypertension', 'Hypotension', 'Tachypnea', 'Bradypnea', 'stroke']


In [4]:
data=data[new_columns]

In [5]:
data

,HADM_ID,SUBJECT_ID,GENDER,AGE,Admission Weight (Kg),Height (cm),Heart Rate,Non Invasive Blood Pressure diastolic,Non Invasive Blood Pressure mean,Non Invasive Blood Pressure systolic,Respiratory Rate,Tachycardia,Bradycardia,Hypertension,Hypotension,Tachypnea,Bradypnea,stroke
0,100018,58128,0,55,125.0,178.0,65.5,70.2,81.8,122.4,14.00,0,0,0,0,0,0,0
1,100018,58128,0,55,125.0,178.0,66.0,70.2,81.8,122.4,15.75,0,0,0,0,0,0,0
2,100018,58128,0,55,125.0,178.0,70.0,70.2,81.8,122.4,15.75,0,0,0,0,0,0,0
3,100018,58128,0,55,125.0,178.0,82.5,69.4,81.6,123.2,15.00,0,0,0,0,0,0,0
4,100018,58128,0,55,125.0,178.0,99.0,69.4,80.8,124.0,17.50,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
207403,178565,46857,1,86,80.0,160.0,69.0,44.6,46.0,127.0,22.00,0,0,0,0,1,0,0
207404,178565,46857,1,86,80.0,160.0,64.0,49.0,73.0,142.0,24.00,0,0,0,0,1,0,0
207405,178565,46857,1,86,80.0,160.0,66.0,43.0,67.0,132.0,16.00,0,0,0,0,0,0,0
207406,178565,46857,1,86,80.0,160.0,67.0,69.6,86.4,140.0,22.00,0,0,0,0,1,0,0


In [6]:
#extractins subset of the top 10 patients with the most measurements
patient_counts = data['SUBJECT_ID'].value_counts()
top_10_patients = patient_counts.nlargest(10).index
subset_df_top10 = data[data['SUBJECT_ID'].isin(top_10_patients)]
subset_df_top10['SUBJECT_ID'].value_counts()

48872    1453
72908    1440
73695    1270
74438    1124
70191    1069
71059    1068
60393     988
70330     949
51385     928
40013     927
Name: SUBJECT_ID, dtype: int64

In [7]:
from sklearn.preprocessing import StandardScaler,RobustScaler
def static_scale(df):
    data=df.copy()
    scaler=RobustScaler()
    static_cols=["AGE",'Admission Weight (Kg)',"Height (cm)"]
    data[static_cols]=scaler.fit_transform(data[static_cols])
    return data,scaler

In [8]:
static_cols=["AGE",'Admission Weight (Kg)',"Height (cm)"]
scaled,scaler=static_scale(subset_df_top10)

In [9]:
from sklearn.preprocessing import StandardScaler,RobustScaler
from tqdm import tqdm
def dynamic_scale(df):
    final_df=pd.DataFrame()
    cols=['Heart Rate',
                  'Non Invasive Blood Pressure diastolic', 'Non Invasive Blood Pressure mean',
                 'Non Invasive Blood Pressure systolic', 'Respiratory Rate']
    for hadm in tqdm(df['HADM_ID'].unique()):
        scaler=RobustScaler()
        #extract the dataframe
        hadm_df=df[df['HADM_ID']==hadm]
        #scale the float cols
        hadm_df[cols]=scaler.fit_transform(hadm_df[cols])
#         print(hadm_df)
        final_df = pd.concat([final_df,hadm_df])
    return final_df

In [10]:
scaled=dynamic_scale(scaled)

100%|██████████| 10/10 [00:00<00:00, 222.97it/s]


In [24]:
#apply this function for eacxh unique admission
#so we won't mix timestamps from different admission in the same training sequence
def create_window(df):
    n_observation=4
    n_forecast=2
    n_target=1
    X,y=[],[]
    for adm_id in df['HADM_ID'].unique():
        data=df[df['HADM_ID']==adm_id]
#         target_cols=["stroke"]
#         target_df=data[target_cols]
    #     data.drop(target_cols,axis=1,inplace=True)
        for i in range(len(data)-6):
            X.append(data.iloc[i:i+n_observation,2:])
            y.append(data.iloc[i+n_observation+n_forecast:i+n_observation+n_forecast+n_target,-1])
    return np.array(X),np.array(y)

In [12]:
neg,pos=np.bincount(subset_df_top10['stroke'])
total=neg+pos
print('Examples:\n    Total: {}\n    Positive: {} ({:.2f}% of total)\n'.format(
    total, pos, 100 * pos / total))
# Scaling by total/2 helps keep the loss to a similar magnitude.
# The sum of the weights of all examples stays the same.
weight_for_0 = (1 / neg) * (total / 2.0)
weight_for_1 = (1 / pos) * (total / 2.0)

class_weight = {0: weight_for_0, 1: weight_for_1}

print('Weight for class 0: {:.2f}'.format(weight_for_0))
print('Weight for class 1: {:.2f}'.format(weight_for_1))

Examples:
    Total: 11216
    Positive: 992 (8.84% of total)

Weight for class 0: 0.55
Weight for class 1: 5.65


In [37]:
# Function to train and evaluate the model based on the provided architecture
from tensorflow import keras
from tensorflow.keras import Input, Model
from tensorflow.keras.layers import Bidirectional, LSTM, Dropout, Dense, Concatenate
from tensorflow.keras.regularizers import l2

def train_model(X_train, y_train, X_val, y_val, class_weight):
    # Define the model architecture
    n_static = 4
    n_timesteps = 4
    n_dynamic = 12
    n_output = 1

    # Define input layer
    recurrent_input = Input(shape=(n_timesteps, n_dynamic), name="TIMESERIES_INPUT")
    static_input = Input(shape=(n_static,), name="STATIC_INPUT")

    # RNN Layers
    # Layer - 1
    rec_layer_one = Bidirectional(LSTM(128, kernel_regularizer=l2(0.01), recurrent_regularizer=l2(0.01), return_sequences=True), name="BIDIRECTIONAL_LAYER_1")(recurrent_input)
    rec_layer_one = Dropout(0.1, name="DROPOUT_LAYER_1")(rec_layer_one)

    # Layer - 2
    rec_layer_two = Bidirectional(LSTM(64, kernel_regularizer=l2(0.01), recurrent_regularizer=l2(0.01)), name="BIDIRECTIONAL_LAYER_2")(rec_layer_one)
    rec_layer_two = Dropout(0.1, name="DROPOUT_LAYER_2")(rec_layer_two)

    # SLP Layers
    static_layer_one = Dense(64, kernel_regularizer=l2(0.001), activation='relu', name="DENSE_LAYER_1")(static_input)

    # Combine layers - RNN + SLP
    combined = Concatenate(axis=1, name="CONCATENATED_TIMESERIES_STATIC")([rec_layer_two, static_layer_one])
    combined_dense_two = Dense(64, activation='relu', name="DENSE_LAYER_2")(combined)
    output = Dense(n_output, activation='sigmoid', name="OUTPUT_LAYER")(combined_dense_two)

    # Compile Model
    model = Model(inputs=[recurrent_input, static_input], outputs=[output])
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=[
        keras.metrics.BinaryAccuracy(name='accuracy'),
        #keras.metrics.Precision(name='precision'),
        keras.metrics.Recall(name='recall')
        #keras.metrics.AUC(name='auc'),
        #keras.metrics.AUC(name='prc', curve='PR')  # precision-recall curve
    ])

    # Extract static features for training and validation sets
    X_train_static = X_train[:, 0, :4]
    X_val_static = X_val[:, 0, :4]

    # Fit the model
    history = model.fit([X_train[:, :, 4:], X_train_static], y_train,
                        epochs=10,
                        batch_size=32,
                        class_weight=class_weight)
    results=model.evaluate([X_val[:, :, 4:], X_val_static],y_val)
    print(results)
    return results




In [14]:
min_hours=subset_df_top10['SUBJECT_ID'].value_counts().min()
n_blocks=5
block_size=int(min_hours/n_blocks)
print(block_size)

185


In [15]:
subset_df_top10[~subset_df_top10['SUBJECT_ID'].isin([40013])]

,HADM_ID,SUBJECT_ID,GENDER,AGE,Admission Weight (Kg),Height (cm),Heart Rate,Non Invasive Blood Pressure diastolic,Non Invasive Blood Pressure mean,Non Invasive Blood Pressure systolic,Respiratory Rate,Tachycardia,Bradycardia,Hypertension,Hypotension,Tachypnea,Bradypnea,stroke
21384,108174,73695,1,44,77.3,168.0,105.0,82.0,85.0,96.0,23.0,1,0,0,0,1,0,0
21385,108174,73695,1,44,77.3,168.0,101.0,57.0,67.0,95.0,21.0,1,0,0,0,1,0,0
21386,108174,73695,1,44,77.3,168.0,88.0,58.0,66.0,87.0,23.0,0,0,0,1,1,0,0
21387,108174,73695,1,44,77.3,168.0,91.0,62.0,68.0,84.0,16.0,0,0,0,0,0,0,0
21388,108174,73695,1,44,77.3,168.0,94.0,60.0,65.0,94.0,16.0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
201921,176846,72908,1,42,67.8,165.0,87.0,83.0,95.0,133.0,29.0,0,0,1,0,1,0,0
201922,176846,72908,1,42,67.8,165.0,90.0,102.0,112.0,150.0,28.0,0,0,1,0,1,0,1
201923,176846,72908,1,42,67.8,165.0,88.0,99.0,118.0,169.0,26.0,0,0,1,0,1,0,1
201924,176846,72908,1,42,67.8,165.0,94.0,70.0,90.0,148.0,29.0,0,0,0,0,1,0,0


In [16]:
def get_val_patient(df):
    counts_df=df['SUBJECT_ID'].value_counts()
    min_index=counts_df.idxmin()
    val_patient=df[df['SUBJECT_ID']==min_index]
    return val_patient,val_patient['SUBJECT_ID'].unique()[0]

In [17]:
subset_df_top10.iloc[0:185]

,HADM_ID,SUBJECT_ID,GENDER,AGE,Admission Weight (Kg),Height (cm),Heart Rate,Non Invasive Blood Pressure diastolic,Non Invasive Blood Pressure mean,Non Invasive Blood Pressure systolic,Respiratory Rate,Tachycardia,Bradycardia,Hypertension,Hypotension,Tachypnea,Bradypnea,stroke
21384,108174,73695,1,44,77.3,168.0,105.0,82.0,85.0,96.0,23.0,1,0,0,0,1,0,0
21385,108174,73695,1,44,77.3,168.0,101.0,57.0,67.0,95.0,21.0,1,0,0,0,1,0,0
21386,108174,73695,1,44,77.3,168.0,88.0,58.0,66.0,87.0,23.0,0,0,0,1,1,0,0
21387,108174,73695,1,44,77.3,168.0,91.0,62.0,68.0,84.0,16.0,0,0,0,0,0,0,0
21388,108174,73695,1,44,77.3,168.0,94.0,60.0,65.0,94.0,16.0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21564,108174,73695,1,44,92.0,168.0,87.0,61.0,67.0,89.0,34.0,0,0,0,0,1,0,0
21565,108174,73695,1,44,92.0,168.0,83.0,68.0,63.0,95.0,31.0,0,0,0,0,1,0,0
21566,108174,73695,1,44,92.0,168.0,84.0,73.0,79.0,97.0,38.0,0,0,0,0,1,0,0
21567,108174,73695,1,44,92.0,168.0,83.0,73.0,78.0,95.0,32.0,0,0,0,0,1,0,0


In [18]:
# Function to compute class weights for imbalanced data, with handling for single-class dataframes
def compute_weights(df, target_column='stroke'):
    bincount_result = np.bincount(df[target_column])
    
    # Handle the case where the DataFrame contains only one class
    if len(bincount_result) == 1:
        if df[target_column].iloc[0] == 0:
            neg = bincount_result[0]
            pos = 0
        else:
            pos = bincount_result[0]
            neg = 0
    else:
        neg, pos = bincount_result
    
    total = neg + pos
    
    # Avoid division by zero when calculating weights
    weight_for_0 = (1 / (neg if neg > 0 else 1)) * (total / 2.0)
    weight_for_1 = (1 / (pos if pos > 0 else 1)) * (total / 2.0)
    
    # Create a dictionary to store the class weights
    class_weight = {0: weight_for_0, 1: weight_for_1}
    
    # Print and return the computed class weights
    print(f'Examples:\n    Total: {total}\n    Positive: {pos} ({100 * pos / total if total > 0 else 0:.2f}% of total)\n')
    print(f'Weight for class 0: {weight_for_0:.2f}')
    print(f'Weight for class 1: {weight_for_1:.2f}')
    
    return class_weight

# Test the function with a DataFrame containing only one class
dummy_df_single_class = pd.DataFrame({'stroke': [0, 0, 0, 0, 0]})
compute_weights(dummy_df_single_class)


Examples:
    Total: 5
    Positive: 0 (0.00% of total)

Weight for class 0: 0.50
Weight for class 1: 2.50


{0: 0.5, 1: 2.5}

In [66]:
def custom_cross_val(df,val_id,n_blocks=5):
    block_metrics=[]
    val_patient=df[df['SUBJECT_ID']==val_id]
    print(val_patient['stroke'].value_counts())
    #calculate the block_size
    min_hours=subset_df_top10['SUBJECT_ID'].value_counts().min()
    block_size=int(min_hours/n_blocks)
    start = 0
    end = block_size
    #print(subset_df_top10['stroke'].iloc[0:740].value_counts())
    #print("#######################")
    #print(f"Cross validation for patient ID: {val_id}")
    for fold in range(n_blocks):
        #initialize the x_train and x_test X_train y_train y_test numpy arrays 
        train_list,label_list=[],[]
        train_df=pd.DataFrame()
        print("#######################")
        print(f"Fold number: {fold + 1}")
        print("#######################")
        #train on all patients and validate on ony one 
        # Determine 80-20 split index within the block
        split_idx = int(block_size * 0.8)
        #print(start, end, split_idx)
        #print(end-(start+split_idx))
        #create the sequence for each patient separately then concatenate
        for p in subset_df_top10['SUBJECT_ID'].unique():
            patient_data=subset_df_top10[subset_df_top10['SUBJECT_ID']==p]
            # Create training subset for this patient
            train = patient_data.iloc[start:start+split_idx]
            #print('len_train_patient ',len(train))
            train_seq, train_label = create_window(train)
            #print(len(train_seq))
            #append to X_train and y_train and train_df
            train_list.append(train_seq)
            label_list.append(train_label)
            train_df=train_df.append(train)
        #append to X_train and y_train
        X_train=np.vstack(train_list)
        y_train=np.vstack(label_list)

        test=subset_df_top10[subset_df_top10['SUBJECT_ID']==val_id][start+split_idx:end]
        #print("train_df length ",len(test))
        X_test, y_test = create_window(test)
        #print("X_test length ",len(X_test))
        print(f"X_train shape: {X_train.shape}, y_train.shape: {y_train.shape}")
        print(f"X_test shape: {X_test.shape}, y_test.shape: {y_test.shape}")
        #print(f"train_df shape: {train_df.shape}")
        weights=compute_weights(train_df[train_df['SUBJECT_ID']==val_id])
        print(weights)
        results = train_model(X_train, y_train, X_test, y_test, weights)
        block_metrics.append({"fold":fold+1,
                             "results":results})
        start = end
        end += block_size
    return block_metrics

In [67]:
def nested_cross_val(df):
    global_metrics=[]
    for idx,val_id in enumerate(df['SUBJECT_ID'].unique()):
        print("##############")
        print(f"iteration {idx+1}")
        print("cross-validation for patient: ",val_id)
        print("##############")
        pat_res=custom_cross_val(df,val_id,n_blocks=5)
        global_metrics.append({"validation patient ID":val_id,
                             "results":pat_res})
    return global_metrics

In [68]:
final_metrics=nested_cross_val(subset_df_top10)

##############
iteration 1
cross-validation for patient:  73695
##############
0    1269
1       1
Name: stroke, dtype: int64
#######################
Fold number: 1
#######################
X_train shape: (1420, 4, 16), y_train.shape: (1420, 1)
X_test shape: (31, 4, 16), y_test.shape: (31, 1)
Examples:
    Total: 148
    Positive: 0 (0.00% of total)

Weight for class 0: 0.50
Weight for class 1: 74.00
{0: 0.5, 1: 74.0}
Epoch 1/10
45/45 [==============================] - 4s 16ms/step - loss: 10.8214 - accuracy: 0.2690 - recall: 0.9216
Epoch 2/10
45/45 [==============================] - 1s 15ms/step - loss: 3.4351 - accuracy: 0.5007 - recall: 0.9902
Epoch 3/10
45/45 [==============================] - 1s 15ms/step - loss: 2.0241 - accuracy: 0.6732 - recall: 0.9902
Epoch 4/10
45/45 [==============================] - 1s 14ms/step - loss: 1.8704 - accuracy: 0.5965 - recall: 0.9902
Epoch 5/10
45/45 [==============================] - 1s 15ms/step - loss: 1.6682 - accuracy: 0.5493 - recall: 0.990

X_train shape: (1420, 4, 16), y_train.shape: (1420, 1)
X_test shape: (31, 4, 16), y_test.shape: (31, 1)
Examples:
    Total: 148
    Positive: 1 (0.68% of total)

Weight for class 0: 0.50
Weight for class 1: 74.00
{0: 0.5034013605442177, 1: 74.0}
Epoch 1/10
45/45 [==============================] - 5s 17ms/step - loss: 9.1906 - accuracy: 0.2972 - recall: 0.9412
Epoch 2/10
45/45 [==============================] - 1s 15ms/step - loss: 3.1418 - accuracy: 0.6268 - recall: 0.9902
Epoch 3/10
45/45 [==============================] - 1s 15ms/step - loss: 2.2336 - accuracy: 0.6951 - recall: 0.9902
Epoch 4/10
45/45 [==============================] - 1s 14ms/step - loss: 2.0290 - accuracy: 0.7056 - recall: 0.9902
Epoch 5/10
45/45 [==============================] - 1s 15ms/step - loss: 1.5658 - accuracy: 0.7444 - recall: 0.9902
Epoch 6/10
45/45 [==============================] - 1s 15ms/step - loss: 1.3662 - accuracy: 0.7423 - recall: 0.9902
Epoch 7/10
45/45 [==============================] - 1s 15

X_train shape: (1420, 4, 16), y_train.shape: (1420, 1)
X_test shape: (31, 4, 16), y_test.shape: (31, 1)
Examples:
    Total: 148
    Positive: 0 (0.00% of total)

Weight for class 0: 0.50
Weight for class 1: 74.00
{0: 0.5, 1: 74.0}
Epoch 1/10
45/45 [==============================] - 4s 17ms/step - loss: 7.2946 - accuracy: 0.4479 - recall: 0.9804
Epoch 2/10
45/45 [==============================] - 1s 15ms/step - loss: 2.8808 - accuracy: 0.6507 - recall: 0.9902
Epoch 3/10
45/45 [==============================] - 1s 15ms/step - loss: 1.5597 - accuracy: 0.8014 - recall: 0.9902
Epoch 4/10
45/45 [==============================] - 1s 15ms/step - loss: 1.9327 - accuracy: 0.5401 - recall: 0.9804
Epoch 5/10
45/45 [==============================] - 1s 15ms/step - loss: 1.3905 - accuracy: 0.6908 - recall: 0.9804
Epoch 6/10
45/45 [==============================] - 1s 14ms/step - loss: 1.3559 - accuracy: 0.6521 - recall: 0.9902
Epoch 7/10
45/45 [==============================] - 1s 15ms/step - loss:

Epoch 1/10
45/45 [==============================] - 4s 16ms/step - loss: 11.9043 - accuracy: 0.2303 - recall: 0.9510
Epoch 2/10
45/45 [==============================] - 1s 14ms/step - loss: 3.2919 - accuracy: 0.6310 - recall: 0.9902
Epoch 3/10
45/45 [==============================] - 1s 14ms/step - loss: 2.2299 - accuracy: 0.6718 - recall: 0.9902
Epoch 4/10
45/45 [==============================] - 1s 15ms/step - loss: 2.1373 - accuracy: 0.6282 - recall: 0.9902
Epoch 5/10
45/45 [==============================] - 1s 14ms/step - loss: 1.4304 - accuracy: 0.8380 - recall: 0.9902
Epoch 6/10
45/45 [==============================] - 1s 14ms/step - loss: 2.3415 - accuracy: 0.4768 - recall: 0.9804
Epoch 7/10
45/45 [==============================] - 1s 15ms/step - loss: 1.6216 - accuracy: 0.6634 - recall: 0.9804
Epoch 8/10
45/45 [==============================] - 1s 15ms/step - loss: 1.2872 - accuracy: 0.7331 - recall: 0.9902
Epoch 9/10
45/45 [==============================] - 1s 15ms/step - loss

45/45 [==============================] - 5s 17ms/step - loss: 7.1563 - accuracy: 0.5197 - recall: 0.9314
Epoch 2/10
45/45 [==============================] - 1s 15ms/step - loss: 2.4269 - accuracy: 0.7930 - recall: 0.9706
Epoch 3/10
45/45 [==============================] - 1s 14ms/step - loss: 1.4475 - accuracy: 0.6937 - recall: 0.9804
Epoch 4/10
45/45 [==============================] - 1s 15ms/step - loss: 1.0102 - accuracy: 0.7894 - recall: 0.9902
Epoch 5/10
45/45 [==============================] - 1s 15ms/step - loss: 0.8579 - accuracy: 0.8113 - recall: 0.9902
Epoch 6/10
45/45 [==============================] - 1s 14ms/step - loss: 1.0778 - accuracy: 0.6859 - recall: 0.9510
Epoch 7/10
45/45 [==============================] - 1s 14ms/step - loss: 0.8036 - accuracy: 0.7782 - recall: 0.9902
Epoch 8/10
45/45 [==============================] - 1s 15ms/step - loss: 0.7560 - accuracy: 0.7866 - recall: 0.9804
Epoch 9/10
45/45 [==============================] - 1s 14ms/step - loss: 0.7191 - a

45/45 [==============================] - 4s 16ms/step - loss: 10.1452 - accuracy: 0.3254 - recall: 0.9608
Epoch 2/10
45/45 [==============================] - 1s 15ms/step - loss: 3.2691 - accuracy: 0.7176 - recall: 0.9902
Epoch 3/10
45/45 [==============================] - 1s 15ms/step - loss: 2.3966 - accuracy: 0.6535 - recall: 0.9902
Epoch 4/10
45/45 [==============================] - 1s 14ms/step - loss: 1.8754 - accuracy: 0.7296 - recall: 0.9902
Epoch 5/10
45/45 [==============================] - 1s 15ms/step - loss: 1.9735 - accuracy: 0.5979 - recall: 0.9902
Epoch 6/10
45/45 [==============================] - 1s 15ms/step - loss: 2.5013 - accuracy: 0.5035 - recall: 0.9706
Epoch 7/10
45/45 [==============================] - 1s 14ms/step - loss: 1.9825 - accuracy: 0.6507 - recall: 0.9706
Epoch 8/10
45/45 [==============================] - 1s 14ms/step - loss: 1.4475 - accuracy: 0.7472 - recall: 0.9902
Epoch 9/10
45/45 [==============================] - 1s 15ms/step - loss: 1.6431 - 

Epoch 1/10
45/45 [==============================] - 4s 15ms/step - loss: 5.6304 - accuracy: 0.9077 - recall: 0.1667
Epoch 2/10
45/45 [==============================] - 1s 15ms/step - loss: 1.3975 - accuracy: 0.9430 - recall: 0.3431
Epoch 3/10
45/45 [==============================] - 1s 14ms/step - loss: 0.4970 - accuracy: 0.9275 - recall: 0.3627
Epoch 4/10
45/45 [==============================] - 1s 15ms/step - loss: 0.2693 - accuracy: 0.9239 - recall: 0.3235
Epoch 5/10
45/45 [==============================] - 1s 14ms/step - loss: 0.1837 - accuracy: 0.9324 - recall: 0.3627
Epoch 6/10
45/45 [==============================] - 1s 14ms/step - loss: 0.1930 - accuracy: 0.9331 - recall: 0.3431
Epoch 7/10
45/45 [==============================] - 1s 14ms/step - loss: 0.1572 - accuracy: 0.9387 - recall: 0.4020
Epoch 8/10
45/45 [==============================] - 1s 14ms/step - loss: 0.2159 - accuracy: 0.9211 - recall: 0.2745
Epoch 9/10
45/45 [==============================] - 1s 14ms/step - loss:

Epoch 1/10
45/45 [==============================] - 5s 16ms/step - loss: 7.7062 - accuracy: 0.3577 - recall: 0.9412
Epoch 2/10
45/45 [==============================] - 1s 14ms/step - loss: 3.4327 - accuracy: 0.5493 - recall: 0.9706
Epoch 3/10
45/45 [==============================] - 1s 14ms/step - loss: 2.0829 - accuracy: 0.7866 - recall: 0.9902
Epoch 4/10
45/45 [==============================] - 1s 15ms/step - loss: 2.1008 - accuracy: 0.4887 - recall: 0.9706
Epoch 5/10
45/45 [==============================] - 1s 15ms/step - loss: 1.6455 - accuracy: 0.6183 - recall: 0.9902
Epoch 6/10
45/45 [==============================] - 1s 15ms/step - loss: 1.3546 - accuracy: 0.7211 - recall: 0.9902
Epoch 7/10
45/45 [==============================] - 1s 15ms/step - loss: 1.4984 - accuracy: 0.6401 - recall: 0.9706
Epoch 8/10
45/45 [==============================] - 1s 15ms/step - loss: 1.6076 - accuracy: 0.6028 - recall: 0.9902
Epoch 9/10
45/45 [==============================] - 1s 15ms/step - loss:

45/45 [==============================] - 4s 17ms/step - loss: 33.9386 - accuracy: 0.2085 - recall: 0.8529
Epoch 2/10
45/45 [==============================] - 1s 15ms/step - loss: 3.9557 - accuracy: 0.3831 - recall: 0.9804
Epoch 3/10
45/45 [==============================] - 1s 15ms/step - loss: 2.2062 - accuracy: 0.6366 - recall: 0.9902
Epoch 4/10
45/45 [==============================] - 1s 15ms/step - loss: 1.9042 - accuracy: 0.6141 - recall: 0.9902
Epoch 5/10
45/45 [==============================] - 1s 14ms/step - loss: 3.2049 - accuracy: 0.3317 - recall: 0.9804
Epoch 6/10
45/45 [==============================] - 1s 15ms/step - loss: 1.3763 - accuracy: 0.5965 - recall: 1.0000
Epoch 7/10
45/45 [==============================] - 1s 15ms/step - loss: 1.4170 - accuracy: 0.6232 - recall: 0.9902
Epoch 8/10
45/45 [==============================] - 1s 15ms/step - loss: 1.3038 - accuracy: 0.6310 - recall: 0.9902
Epoch 9/10
45/45 [==============================] - 1s 15ms/step - loss: 1.3325 - 

45/45 [==============================] - 4s 16ms/step - loss: 6.8240 - accuracy: 0.7915 - recall: 0.5686
Epoch 2/10
45/45 [==============================] - 1s 14ms/step - loss: 1.6833 - accuracy: 0.9176 - recall: 0.8333
Epoch 3/10
45/45 [==============================] - 1s 15ms/step - loss: 0.6558 - accuracy: 0.9070 - recall: 0.8725
Epoch 4/10
45/45 [==============================] - 1s 15ms/step - loss: 0.3731 - accuracy: 0.9289 - recall: 0.8333
Epoch 5/10
45/45 [==============================] - 1s 15ms/step - loss: 0.2973 - accuracy: 0.9000 - recall: 0.8431
Epoch 6/10
45/45 [==============================] - 1s 15ms/step - loss: 0.2591 - accuracy: 0.9035 - recall: 0.8333
Epoch 7/10
45/45 [==============================] - 1s 15ms/step - loss: 0.2626 - accuracy: 0.9176 - recall: 0.8725
Epoch 8/10
45/45 [==============================] - 1s 16ms/step - loss: 0.2340 - accuracy: 0.9169 - recall: 0.8725
Epoch 9/10
45/45 [==============================] - 1s 14ms/step - loss: 0.2809 - a

In [70]:
for metric_dic in final_metrics:
    for k,v in metric_dic.items():
        print(k)
        print(v)
    

validation patient ID
73695
results
[{'fold': 1, 'results': [4.418652534484863, 0.0, 0.0]}, {'fold': 2, 'results': [0.37805941700935364, 1.0, 0.0]}, {'fold': 3, 'results': [2.50274920463562, 0.0, 0.0]}, {'fold': 4, 'results': [0.3759647607803345, 1.0, 0.0]}, {'fold': 5, 'results': [0.6404802799224854, 1.0, 0.0]}]
validation patient ID
71059
results
[{'fold': 1, 'results': [5.760497093200684, 0.0, 0.0]}, {'fold': 2, 'results': [0.6409904360771179, 0.9677419066429138, 0.0]}, {'fold': 3, 'results': [2.6720709800720215, 0.0, 0.0]}, {'fold': 4, 'results': [0.7266538739204407, 0.6774193644523621, 0.0]}, {'fold': 5, 'results': [0.2965176999568939, 0.9677419066429138, 0.0]}]
validation patient ID
48872
results
[{'fold': 1, 'results': [0.19342175126075745, 1.0, 0.0]}, {'fold': 2, 'results': [0.270153284072876, 1.0, 0.0]}, {'fold': 3, 'results': [2.2693934440612793, 0.0, 0.0]}, {'fold': 4, 'results': [1.479731798171997, 0.29032257199287415, 0.0]}, {'fold': 5, 'results': [1.1136205196380615, 0.03

In [99]:
final_metrics[0]['results'][0]['results']

[4.418652534484863, 0.0, 0.0]

In [103]:
average_results = {}

for metric_dict in final_metrics:
    
    loss = []
    acc = []
    recall = []  
    # Loop through each fold's results
    for fold in metric_dict['results']:
        results = fold['results']
        loss.append(results[0])
        acc.append(results[1])
        recall.append(results[2])

    # Calculate average for each metric across all folds
    avg_loss = np.mean(loss)
    avg_acc = np.mean(acc)
    avg_recall = np.mean(recall)

    # Store average results for this validation patient ID
    average_results[metric_dict['validation patient ID']] = [avg_loss, avg_acc, avg_recall]

average_results


{73695: [1.6631812393665313, 0.6, 0.0],
 71059: [2.0193460166454313, 0.5225806355476379, 0.0],
 48872: [1.0652641594409942, 0.4645161271095276, 0.0],
 40013: [0.5590966939926147, 0.8451612889766693, 0.0],
 60393: [0.39842030107975007, 0.8129032194614411, 0.0],
 70330: [0.4796446353197098, 0.8193548321723938, 0.2],
 74438: [0.7917275905609131, 0.6774193465709686, 0.6],
 51385: [0.5824076861143113, 0.8580645084381103, 0.0],
 70191: [0.7392233699560166, 0.7870967745780945, 0.0],
 72908: [0.7280100345611572, 0.6903225839138031, 0.0]}

In [110]:
acc=[]
rec=[]
for metrics in average_results.values():
    acc.append(metrics[1])
print(np.mean(acc))
    

0.7077419316768645


- error analysis
1. iteration 1
cross-validation for patient 73695:  

0    1269

1       1

2. iteration 2
cross-validation for patient 71059:  

0    1054

1      14

3. iteration 3
cross-validation for patient 48872:  

0    1443

1      10

4. iteration 4
cross-validation for patient 40013:  

0    926

1      1

5. iteration 5
cross-validation for patient 60393:  

0    985

1      3

6. iteration 6
cross-validation for patient 70330:  

0    939

1     10

7. iteration 7
cross-validation for patient 74438:  

1    567

0    557

8. iteration 8
cross-validation for patient 51385:  

0    927

1      1

9. iteration 9
cross-validation for patient 70191:  

0    1067

1       2

10. iteration 10
cross-validation for patient 72908:  

0    1057

1     383
 


## computing weights across the entire stack of patients

In [134]:
def custom_cross_val_v2(df,val_id,n_blocks=5):
    block_metrics=[]
    val_patient=df[df['SUBJECT_ID']==val_id]
    print(val_patient['stroke'].value_counts())
    #calculate the block_size
    min_hours=subset_df_top10['SUBJECT_ID'].value_counts().min()
    block_size=int(min_hours/n_blocks)
    start = 0
    end = block_size
    #print(subset_df_top10['stroke'].iloc[0:740].value_counts())
    #print("#######################")
    #print(f"Cross validation for patient ID: {val_id}")
    for fold in range(n_blocks):
        #initialize the x_train and x_test X_train y_train y_test numpy arrays 
        train_list,label_list=[],[]
        train_df=pd.DataFrame()
        print("#######################")
        print(f"Fold number: {fold + 1}")
        print("#######################")
        print("start pointer: ",start)
        print("end pointer; ",end)
        #train on all patients and validate on ony one 
        # Determine 80-20 split index within the block
        split_idx = int(block_size * 0.8)
        #print(start, end, split_idx)
        #print(end-(start+split_idx))
        #create the sequence for each patient separately then concatenate
        for p in subset_df_top10['SUBJECT_ID'].unique():
            patient_data=subset_df_top10[subset_df_top10['SUBJECT_ID']==p]
            # Create training subset for this patient
            train = patient_data.iloc[start:start+split_idx]
            #print('len_train_patient ',len(train))
            train_seq, train_label = create_window(train)
            #print(len(train_seq))
            #append to X_train and y_train and train_df
            train_list.append(train_seq)
            label_list.append(train_label)
            train_df=train_df.append(train)
        #append to X_train and y_train
        X_train=np.vstack(train_list)
        y_train=np.vstack(label_list)

        test=subset_df_top10[subset_df_top10['SUBJECT_ID']==val_id][start+split_idx:end]
        #print("train_df length ",len(test))
        X_test, y_test = create_window(test)
        #print("X_test length ",len(X_test))
        print(f"X_train shape: {X_train.shape}, y_train.shape: {y_train.shape}")
        print(f"X_test shape: {X_test.shape}, y_test.shape: {y_test.shape}")
        #print(f"train_df shape: {train_df.shape}")
        print("class distribution for train_df")
        print(compute_weights(train_df))
        print("class distribution for test_df")
        print(compute_weights(test))
        results = train_model(X_train, y_train, X_test, y_test, compute_weights(train_df))
        block_metrics.append({"fold":fold+1,
                             "results":results})
        start = end
        end += block_size
    return block_metrics

In [135]:
def nested_cross_val_2(df):
    global_metrics=[]
    for idx,val_id in enumerate(df['SUBJECT_ID'].unique()):
        print("##############")
        print(f"iteration {idx+1}")
        print("cross-validation for patient: ",val_id)
        print("##############")
        pat_res=custom_cross_val_v2(df,val_id,n_blocks=5)
        global_metrics.append({"validation patient ID":val_id,
                             "results":pat_res})
    return global_metrics

In [136]:
final_metrics_v2=nested_cross_val_2(subset_df_top10)

##############
iteration 1
cross-validation for patient:  73695
##############
0    1269
1       1
Name: stroke, dtype: int64
#######################
Fold number: 1
#######################
start pointer:  0
end pointer;  185
X_train shape: (1420, 4, 16), y_train.shape: (1420, 1)
X_test shape: (31, 4, 16), y_test.shape: (31, 1)
class distribution for train_df
Examples:
    Total: 1480
    Positive: 107 (7.23% of total)

Weight for class 0: 0.54
Weight for class 1: 6.92
{0: 0.538965768390386, 1: 6.915887850467289}
class distribution for test_df
Examples:
    Total: 37
    Positive: 0 (0.00% of total)

Weight for class 0: 0.50
Weight for class 1: 18.50
{0: 0.5, 1: 18.5}
Examples:
    Total: 1480
    Positive: 107 (7.23% of total)

Weight for class 0: 0.54
Weight for class 1: 6.92
Epoch 1/10
45/45 [==============================] - 5s 16ms/step - loss: 5.9564 - accuracy: 0.8070 - recall: 0.8137
Epoch 2/10
45/45 [==============================] - 1s 14ms/step - loss: 1.6171 - accuracy: 0.87

Epoch 1/10
45/45 [==============================] - 5s 16ms/step - loss: 6.5238 - accuracy: 0.7782 - recall: 0.7317
Epoch 2/10
45/45 [==============================] - 1s 14ms/step - loss: 1.8205 - accuracy: 0.8662 - recall: 0.8862
Epoch 3/10
45/45 [==============================] - 1s 14ms/step - loss: 0.7434 - accuracy: 0.8768 - recall: 0.9024
Epoch 4/10
45/45 [==============================] - 1s 14ms/step - loss: 0.5312 - accuracy: 0.8655 - recall: 0.8699
Epoch 5/10
45/45 [==============================] - 1s 14ms/step - loss: 0.3944 - accuracy: 0.8782 - recall: 0.9431
Epoch 6/10
45/45 [==============================] - 1s 14ms/step - loss: 0.3467 - accuracy: 0.8754 - recall: 0.9350
Epoch 7/10
45/45 [==============================] - 1s 15ms/step - loss: 0.3183 - accuracy: 0.8782 - recall: 0.9675
Epoch 8/10
45/45 [==============================] - 1s 15ms/step - loss: 0.3298 - accuracy: 0.8718 - recall: 0.9756
Epoch 9/10
45/45 [==============================] - 1s 15ms/step - loss:

45/45 [==============================] - 1s 16ms/step - loss: 1.8543 - accuracy: 0.8127 - recall: 0.8136
Epoch 3/10
45/45 [==============================] - 1s 15ms/step - loss: 0.8848 - accuracy: 0.8155 - recall: 0.8729
Epoch 4/10
45/45 [==============================] - 1s 15ms/step - loss: 0.5957 - accuracy: 0.8359 - recall: 0.8814
Epoch 5/10
45/45 [==============================] - 1s 15ms/step - loss: 0.4967 - accuracy: 0.8352 - recall: 0.8983
Epoch 6/10
45/45 [==============================] - 1s 15ms/step - loss: 0.4897 - accuracy: 0.8317 - recall: 0.9153
Epoch 7/10
45/45 [==============================] - 1s 15ms/step - loss: 0.4651 - accuracy: 0.8479 - recall: 0.8729
Epoch 8/10
45/45 [==============================] - 1s 15ms/step - loss: 0.4501 - accuracy: 0.8394 - recall: 0.9237
Epoch 9/10
45/45 [==============================] - 1s 15ms/step - loss: 0.4501 - accuracy: 0.8197 - recall: 0.9068
Epoch 10/10
1/1 [==============================] - 1s 953ms/step - loss: 0.3893 - a

45/45 [==============================] - 1s 14ms/step - loss: 0.9416 - accuracy: 0.8331 - recall: 0.9250
Epoch 4/10
45/45 [==============================] - 1s 16ms/step - loss: 0.6563 - accuracy: 0.8338 - recall: 0.8625
Epoch 5/10
45/45 [==============================] - 1s 15ms/step - loss: 0.5076 - accuracy: 0.8507 - recall: 0.9500
Epoch 6/10
45/45 [==============================] - 1s 14ms/step - loss: 0.5340 - accuracy: 0.8225 - recall: 0.8500
Epoch 7/10
45/45 [==============================] - 1s 14ms/step - loss: 0.5119 - accuracy: 0.8444 - recall: 0.8625
Epoch 8/10
45/45 [==============================] - 1s 15ms/step - loss: 0.4713 - accuracy: 0.8176 - recall: 0.9125
Epoch 9/10
45/45 [==============================] - 1s 15ms/step - loss: 0.5021 - accuracy: 0.8423 - recall: 0.8500
Epoch 10/10
1/1 [==============================] - 1s 898ms/step - loss: 0.3217 - accuracy: 1.0000 - recall: 0.0000e+00
[0.32168757915496826, 1.0, 0.0]
#######################
Fold number: 4
########

45/45 [==============================] - 1s 14ms/step - loss: 0.5516 - accuracy: 0.8803 - recall: 0.9018
Epoch 5/10
45/45 [==============================] - 1s 15ms/step - loss: 0.5599 - accuracy: 0.8570 - recall: 0.8661
Epoch 6/10
45/45 [==============================] - 1s 15ms/step - loss: 0.4230 - accuracy: 0.8944 - recall: 0.9196
Epoch 7/10
45/45 [==============================] - 1s 15ms/step - loss: 0.3585 - accuracy: 0.8761 - recall: 0.9821
Epoch 8/10
45/45 [==============================] - 1s 14ms/step - loss: 0.4615 - accuracy: 0.8585 - recall: 0.8839
Epoch 9/10
45/45 [==============================] - 1s 16ms/step - loss: 0.3676 - accuracy: 0.8782 - recall: 0.9554
Epoch 10/10
1/1 [==============================] - 1s 907ms/step - loss: 0.0596 - accuracy: 1.0000 - recall: 0.0000e+00
[0.0596129447221756, 1.0, 0.0]
#######################
Fold number: 3
#######################
start pointer:  370
end pointer;  555
X_train shape: (1420, 4, 16), y_train.shape: (1420, 1)
X_test s

...

45/45 [==============================] - 1s 15ms/step - loss: 0.4410 - accuracy: 0.8366 - recall: 0.9000
Epoch 10/10
1/1 [==============================] - 1s 920ms/step - loss: 3.5376 - accuracy: 0.2903 - recall: 1.0000
[3.5375757217407227, 0.29032257199287415, 1.0]
#######################
Fold number: 4
#######################
start pointer:  555
end pointer;  740
X_train shape: (1420, 4, 16), y_train.shape: (1420, 1)
X_test shape: (31, 4, 16), y_test.shape: (31, 1)
class distribution for train_df
Examples:
    Total: 1480
    Positive: 124 (8.38% of total)

Weight for class 0: 0.55
Weight for class 1: 5.97
{0: 0.5457227138643068, 1: 5.967741935483871}
class distribution for test_df
Examples:
    Total: 37
    Positive: 14 (37.84% of total)

Weight for class 0: 0.80
Weight for class 1: 1.32
{0: 0.8043478260869565, 1: 1.3214285714285714}
Examples:
    Total: 1480
    Positive: 124 (8.38% of total)

Weight for class 0: 0.55
Weight for class 1: 5.97
Epoch 1/10
45/45 [===================

1/1 [==============================] - 1s 1s/step - loss: 0.0560 - accuracy: 1.0000 - recall: 0.0000e+00
[0.056035444140434265, 1.0, 0.0]
#######################
Fold number: 3
#######################
start pointer:  370
end pointer;  555
X_train shape: (1420, 4, 16), y_train.shape: (1420, 1)
X_test shape: (31, 4, 16), y_test.shape: (31, 1)
class distribution for train_df
Examples:
    Total: 1480
    Positive: 88 (5.95% of total)

Weight for class 0: 0.53
Weight for class 1: 8.41
{0: 0.5316091954022989, 1: 8.40909090909091}
class distribution for test_df
Examples:
    Total: 37
    Positive: 0 (0.00% of total)

Weight for class 0: 0.50
Weight for class 1: 18.50
{0: 0.5, 1: 18.5}
Examples:
    Total: 1480
    Positive: 88 (5.95% of total)

Weight for class 0: 0.53
Weight for class 1: 8.41
Epoch 1/10
45/45 [==============================] - 4s 17ms/step - loss: 7.2098 - accuracy: 0.6535 - recall: 0.5750
Epoch 2/10
45/45 [==============================] - 1s 15ms/step - loss: 1.9687 - ac

1/1 [==============================] - 1s 1s/step - loss: 0.5877 - accuracy: 0.9355 - recall: 0.0000e+00
[0.5876544713973999, 0.9354838728904724, 0.0]
#######################
Fold number: 2
#######################
start pointer:  185
end pointer;  370
X_train shape: (1420, 4, 16), y_train.shape: (1420, 1)
X_test shape: (31, 4, 16), y_test.shape: (31, 1)
class distribution for train_df
Examples:
    Total: 1480
    Positive: 117 (7.91% of total)

Weight for class 0: 0.54
Weight for class 1: 6.32
{0: 0.5429200293470287, 1: 6.324786324786325}
class distribution for test_df
Examples:
    Total: 37
    Positive: 0 (0.00% of total)

Weight for class 0: 0.50
Weight for class 1: 18.50
{0: 0.5, 1: 18.5}
Examples:
    Total: 1480
    Positive: 117 (7.91% of total)

Weight for class 0: 0.54
Weight for class 1: 6.32
Epoch 1/10
45/45 [==============================] - 5s 17ms/step - loss: 6.1765 - accuracy: 0.7106 - recall: 0.7143
Epoch 2/10
45/45 [==============================] - 1s 15ms/step - l

X_train shape: (1420, 4, 16), y_train.shape: (1420, 1)
X_test shape: (31, 4, 16), y_test.shape: (31, 1)
class distribution for train_df
Examples:
    Total: 1480
    Positive: 107 (7.23% of total)

Weight for class 0: 0.54
Weight for class 1: 6.92
{0: 0.538965768390386, 1: 6.915887850467289}
class distribution for test_df
Examples:
    Total: 37
    Positive: 18 (48.65% of total)

Weight for class 0: 0.97
Weight for class 1: 1.03
{0: 0.9736842105263157, 1: 1.0277777777777777}
Examples:
    Total: 1480
    Positive: 107 (7.23% of total)

Weight for class 0: 0.54
Weight for class 1: 6.92
Epoch 1/10
45/45 [==============================] - 4s 17ms/step - loss: 6.5618 - accuracy: 0.6606 - recall: 0.6765
Epoch 2/10
45/45 [==============================] - 1s 15ms/step - loss: 1.7412 - accuracy: 0.8775 - recall: 0.9510
Epoch 3/10
45/45 [==============================] - 1s 15ms/step - loss: 0.7526 - accuracy: 0.8613 - recall: 0.9412
Epoch 4/10
45/45 [==============================] - 1s 14ms

Epoch 1/10
45/45 [==============================] - 6s 17ms/step - loss: 6.0721 - accuracy: 0.7373 - recall: 0.7805
Epoch 2/10
45/45 [==============================] - 1s 15ms/step - loss: 1.8021 - accuracy: 0.8549 - recall: 0.8943
Epoch 3/10
45/45 [==============================] - 1s 15ms/step - loss: 0.8337 - accuracy: 0.8451 - recall: 0.8537
Epoch 4/10
45/45 [==============================] - 1s 15ms/step - loss: 0.5361 - accuracy: 0.8718 - recall: 0.9024
Epoch 5/10
45/45 [==============================] - 1s 15ms/step - loss: 0.3817 - accuracy: 0.8739 - recall: 0.9675
Epoch 6/10
45/45 [==============================] - 1s 15ms/step - loss: 0.3550 - accuracy: 0.8810 - recall: 0.9675
Epoch 7/10
45/45 [==============================] - 1s 15ms/step - loss: 0.3206 - accuracy: 0.8789 - recall: 0.9675
Epoch 8/10
45/45 [==============================] - 1s 15ms/step - loss: 0.3278 - accuracy: 0.8775 - recall: 0.9756
Epoch 9/10
45/45 [==============================] - 1s 15ms/step - loss:

In [127]:
average_results_2 = {}

for metric_dict in final_metrics_v2:
    
    loss = []
    acc = []
    recall = []  
    # Loop through each fold's results
    for fold in metric_dict['results']:
        results = fold['results']
        loss.append(results[0])
        acc.append(results[1])
        recall.append(results[2])

    # Calculate average for each metric across all folds
    avg_loss = np.mean(loss)
    avg_acc = np.mean(acc)
    avg_recall = np.mean(recall)

    # Store average results for this validation patient ID
    average_results_2[metric_dict['validation patient ID']] = [avg_loss, avg_acc, avg_recall]

average_results_2


{73695: [0.08168927133083344, 1.0, 0.0],
 71059: [0.49174468517303466, 0.7935483813285827, 0.0],
 48872: [0.07214270606637001, 1.0, 0.0],
 40013: [0.43570808619260787, 0.8193548381328583, 0.0],
 60393: [0.0496336467564106, 1.0, 0.0],
 70330: [0.20879362821578978, 0.9225806355476379, 0.0],
 74438: [2.0265417814254763, 0.4451612770557404, 0.9799999952316284],
 51385: [0.08599675446748734, 1.0, 0.0],
 70191: [0.15193893611431122, 0.9870967745780945, 0.0],
 72908: [1.1356263637542725, 0.5032258033752441, 0.6]}

In [128]:
avg_acc=np.mean([metrics[1] for metrics in average_results_2.values()])

In [129]:
res

0.8412903189659119